In [1]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180

import numpy as np
import datetime as dt

import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
#from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
#from statsmodels import robust
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches
import os
import matplotlib.cm as cm

# run pca and visualize

import scipy

colors = [
'black','blue','red','green','cyan','magenta','brown','pink',
'orange','firebrick','lawngreen','dodgerblue','crimson','orchid','slateblue',
'darkgreen','darkorange','indianred','darkviolet','deepskyblue','greenyellow',
'peru','cadetblue','forestgreen','slategrey','lightsteelblue','rebeccapurple',
'darkmagenta','yellow','hotpink']

sorted_colors=colors

#import deconv

# visualize recomputed templates over time;
def binary_reader_waveforms(filename, n_channels, n_times, spikes, channels=None, data_type='float32'):
    ''' Reader for loading raw binaries
    
        standardized_filename:  name of file contianing the raw binary
        n_channels:  number of channels in the raw binary recording 
        n_times:  length of waveform 
        spikes: 1D array containing spike times in sample rate of raw data
        channels: load specific channels only
        data_type: float32 for standardized data
    
    '''

    # ***** LOAD RAW RECORDING *****
    if channels is None:
        wfs = np.zeros((spikes.shape[0], n_times, n_channels), data_type)
    else:
        wfs = np.zeros((spikes.shape[0], n_times, channels.shape[0]), data_type)
    if data_type =='float32':
        data_len = 4
    else:
        data_len = 2

    with open(filename, "rb") as fin:
        for ctr,s in enumerate(spikes):
            #print (ctr,s)
            # index into binary file: time steps * 4  4byte floats * n_channels
            fin.seek(s * data_len * n_channels, os.SEEK_SET)
            wfs[ctr] = np.fromfile(
                fin,
                dtype=data_type,
                count=(n_times * n_channels)).reshape(n_times, n_channels)[:,channels]
    fin.close()
    return wfs

Autosaving every 180 seconds


In [64]:
data = np.load('/media/cat/12TB/insync_cm3746/paninski_lab/data/neuropixels/ari/julien_real_neighbour_chans_neuropixels2/chan_5_data.npy')
templates = np.load('/media/cat/12TB/insync_cm3746/paninski_lab/data/neuropixels/ari/julien_real_neighbour_chans_neuropixels2/chan_5_templates.npy',allow_pickle=True)
times = np.load('/media/cat/12TB/insync_cm3746/paninski_lab/data/neuropixels/ari/julien_real_neighbour_chans_neuropixels2/chan_5_times.npy',allow_pickle=True)
plt.plot(data[:10000])
for p in range(templates.shape[0]):
    idx = np.where(times[p]<10000)[0]
    print (idx.shape)
    for k in range(idx.shape[0]):
        plt.plot(times[p][k]+np.arange(75)-9,templates[p],c=colors[p])
plt.show()

(0,)
(0,)
(3,)
(0,)
(0,)
(0,)
(0,)
(0,)
(1,)
(2,)
(1,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(2,)
(3,)
(0,)
(0,)
(0,)
(2,)


In [2]:
# YASS DATASETS
# load raw data and spike train
root_dir = '/media/cat/12TB/insync_cm3746/paninski_lab/data/retina/512chan/ari_Julien_CMP/2009_real_retina_data/data/'

temps = np.load('/media/cat/2TB/liam/512channels/2009-04-13-5_120mins/tmp/output/templates/templates_0sec.npy')
print (temps.shape)

geom = np.loadtxt('/media/cat/2TB/liam/512channels/2009-04-13-5_120mins/geom.txt')


data = np.fromfile('/media/cat/2TB/liam/512channels/2009-04-13-5_120mins/tmp/preprocess/standardized.bin','float32')
print (data.shape)

n_chan = 512
data2D = data.reshape(-1,n_chan)
print (data2D.shape)

spike_train = np.load('/media/cat/2TB/liam/512channels/2009-04-13-5_120mins/tmp/output/spike_train.npy')
print (spike_train)

(3072000000,)
(6000000, 512)
[[        0      1430]
 [        0       490]
 [        0       565]
 ...
 [143999995       239]
 [143999996       812]
 [143999997        17]]


In [8]:
# save channels with > 3 templates on them
ptps = temps.ptp(1).max(1)
max_chans = temps.ptp(1).argmax(1)


root_dir = '/media/cat/12TB/insync_cm3746/paninski_lab/data/retina/512chan/ari_Julien_CMP/2009_real_retina_data/data/'


ctr=0
for chan in range(n_chan):
    idx2 = np.where(max_chans==chan)[0]
    ptps_local = ptps[idx2]

    locs = geom[chan]
    
    thresh=70
    
    chans_neighbour = []
    for k in range(n_chan):
        dist = np.linalg.norm(locs-geom[k])
        if dist < thresh:
            chans_neighbour.append(k)

            
    #print (chans_neighbour)
    #break
    # all chans within neighbourhood
    chans_neighbour = np.unique(chans_neighbour)
       
    #idx3 = np.where(ptps_local>8.0)[0]
    
    
    if idx2.shape[0]>5:
                
        # save this channel and the data
        np.save(root_dir+'/chan_'+str(ctr)+'_data.npy',
                data2D[:,chan])
        
        # save templates
        temps_out=[]
        for ch in chans_neighbour:
            idx = np.where(max_chans==ch)[0]
            temps_out.extend(temps[idx,:,chan])  #Make sure you save max channel tempalte NOT 
            
        temps_out = np.array(temps_out)[:,20:95]
        print (" templates saved: ", temps_out.shape)
            
        np.save(root_dir+'/chan_'+str(ctr)+'_templates.npy',
                temps_out)
        
        # save spiketrains
        times_out= []
        for ch in chans_neighbour:
            #idx = np.where
            ids = np.where(max_chans==ch)[0]
            for id_ in ids:
                idx4 = np.where(spike_train[:,1]==id_)[0]
                times = spike_train[idx4,0]
                idx5 = np.where(times<(5*60*20000))[0]
                times_out.append(times[idx5])
        
        np.save(root_dir+'/chan_'+str(ctr)+'_times.npy',
                times_out)
        #break    
        ctr+=1

 templates saved:  (29, 75)
 templates saved:  (22, 75)
 templates saved:  (21, 75)
 templates saved:  (21, 75)
 templates saved:  (25, 75)
 templates saved:  (19, 75)
 templates saved:  (49, 75)
 templates saved:  (52, 75)
 templates saved:  (59, 75)
 templates saved:  (53, 75)
 templates saved:  (61, 75)
 templates saved:  (53, 75)


FileNotFoundError: [Errno 2] No such file or directory: '/media/cat/12TB/insync_cm3746/paninski_lab/data/retina/512chan/ari_Julien_CMP/2009_real_retina_data/data//chan_11_times.npy'

In [20]:
data = np.fromfile('/media/cat/2TB/liam/512channels/2009-04-13-5_120mins/2009-04-13-5.bin','int16',count=512*20000*5*60)
print (data.shape)

(3072000000,)


In [22]:
data.tofile('/media/cat/2TB/liam/512channels/2009-04-13-5_120mins/2009-04-13-5_5min.bin')